## *import libraries*

In [1]:
import tqdm

# Patch tqdm to disable progress bars globally
class HiddenTqdm(tqdm.tqdm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs, disable=True)

tqdm.tqdm = HiddenTqdm  # Replace tqdm with the modified class

In [2]:
import functions as f

import os
import re
import numpy as np
import pandas as pd
from tabulate import tabulate
from scipy.spatial.distance import cdist

import pm4py
from pm4py.algo.evaluation import algorithm
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.algo.conformance.tokenreplay import algorithm as token_replay
from pm4py.objects.petri_net.importer.variants import pnml as pnml_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments

import warnings
warnings.filterwarnings('ignore')

# import tqdm
# tqdm.tqdm.__init__ = lambda *args, **kwargs: None

## *Parameter Setting*

## *Read Log and Discovered Routines*

In [3]:
def UI_read_log(log_path):
    # Load event log
    log = f.read_log(log_path)
    
    # Define activities in the reference set
    reference_set = list(log['concept:name'].unique())
    
    # print("\n")
    # print("="*150)
    # print(f'All Activities of the Event Log: \n{reference_set}')
    # print("="*150, "\n")

    return log, reference_set

In [4]:
def read_routines(routines_path):
    all_routines = []
    # Open the text file and read line by line
    with open(routines_path, "r") as file:
        for line in file:
            # Use regex to extract the pattern between square brackets
            match = re.search(r"\[(.*?)\]", line)
            
            if match:
                # Split the pattern by commas to create a list of events
                routine = [event.strip() for event in match.group(1).split(',')]
                # Add the routine to the main list
                all_routines.append(routine)
    return all_routines

## *Prepare encoding of discovered routines*

In [5]:
def get_discover_routines_encoding(all_routines, reference_set):
    routines = dict()
    
    # Iterate over the sequences
    for i, sequence in enumerate(all_routines):
        # Define a unique routine name for each sequence
        routine_name = f"pattern{i+1}"
        # Convert the sequence into a set of unique activities
        routines[routine_name] = sequence
    
    routine_vectors = {name: [1 if act in routine else 0 for act in reference_set] for name, routine in routines.items()}
    return routine_vectors, routines

## *Prepare encoding of traces*

In [6]:
def freq_encoding(session_log):
    # Perform one-hot encoding
    one_hot_encoded = pd.get_dummies(session_log['concept:name'])
    
    # Replace frequency with 1 where frequency is not 0
    one_hot_encoded = one_hot_encoded.applymap(lambda x: 1 if x > 0 else 0)
    
    if 'lifecycle:transition' in session_log.columns:
        df_encoded = pd.concat([session_log[['case:concept:name', 'lifecycle:transition', 'Session']], one_hot_encoded], axis=1)
        df_grouped = df_encoded.groupby(['case:concept:name', 'lifecycle:transition', 'Session']).sum().reset_index()
    else:
        df_encoded = pd.concat([session_log[['case:concept:name', 'Session']], one_hot_encoded], axis=1)
        df_grouped = df_encoded.groupby(['case:concept:name', 'Session']).sum().reset_index()

    return df_grouped


def assign_labels_by_distance(encoded_log, encoded_vectors, reference_set):

    # Convert the encoded vectors to a DataFrame for easier computation
    vector_labels = list(encoded_vectors.keys())
    vector_values = np.array(list(encoded_vectors.values()))
    
    # Compute distances for each row in the log
    enc_log = encoded_log[reference_set]
    distances = np.linalg.norm(enc_log.values[:, None] - vector_values, axis=2)
    
    # Find the index of the minimum distance for each sample
    min_distance_indices = np.argmin(distances, axis=1)
    
    # Map the indices to vector labels
    assigned_labels = [vector_labels[idx] for idx in min_distance_indices]
    
    # Add the assigned labels to the log
    encoded_log['pattern_label'] = assigned_labels
    return encoded_log

## *Assign Patterns to Traces*

In [7]:
def assign_pattern_to_traces(session_log, cluster_log):

    cluster_column = 'pattern_label'


    if 'lifecycle:transition' in session_log.columns:
        # Perform inner join
        merged_log = pd.merge(session_log, cluster_log, on=['case:concept:name', 'Session', 'lifecycle:transition'], how='inner')
        merged_log = merged_log[['case:concept:name', 'Session', 'lifecycle:transition', 'time:timestamp', 'concept:name', cluster_column]]
    else:
        # Perform inner join
        merged_log = pd.merge(session_log, cluster_log, on=['case:concept:name', 'Session'], how='inner')
        merged_log = merged_log[['case:concept:name', 'Session', 'time:timestamp', 'concept:name', cluster_column]]

    merged_log = merged_log.sort_values(by=['case:concept:name', 'Session', 'time:timestamp'])

    activity_log = merged_log[['case:concept:name', 'time:timestamp','concept:name', cluster_column]]

    return activity_log

## *Evaluation Functions*

In [8]:
# Extract transitions (activities) from both models to calculate Jaccard Coefficient
def extract_transitions(net):
    """ Extracts the transitions (activities) from the given Petri net model. """
    return {t.label for t in net.transitions if t.label is not None}


def get_fScore(fitness, precision):
    f_score = (2*fitness*precision)/(fitness+precision)
    return f_score


def get_JC(net, im, fm, routine_label, routines):
    gt_routine_activities = extract_transitions(net)
    routine_activities = set(routines[routine_label])
    # print(f"Ground Truth Activities: {gt_routine_activities}\n")
    # print(f"Discovered Activities: {routine_activities}\n")

    # Calculate the Jaccard Coefficient (Intersection over Union)
    intersection = routine_activities.intersection(gt_routine_activities)
    union = routine_activities.union(gt_routine_activities)
    jaccard_coefficient = len(intersection) / len(union)

    return jaccard_coefficient


def token_base_evaluation(log, net, im, fm):
    replayed_traces = token_replay.apply(log, net, im, fm)
    
    # Calculate Support (fraction of transitions supported by the log)
    activated_transitions = set()
    for trace in replayed_traces:
        for trans in trace['activated_transitions']:
            if trans is not None:  # Ensure the transition is valid
                activated_transitions.add(trans)
    total_transitions = len(net.transitions)
    support = len(activated_transitions) / total_transitions

    # Calculate Coverage
    covered_traces = sum([1 for trace in replayed_traces if trace['activated_transitions']])
    total_traces = len(log)
    coverage = covered_traces / total_traces

    return support, coverage
    

def alignment_base_evaluation(log, net, im, fm):
    # Apply alignment-based conformance checking
    aligned_traces = alignments.apply_log(log, net, im, fm)
    
    # Calculate Support (fraction of transitions supported by the log)
    activated_transitions = set()
    for trace in aligned_traces:
        for step in trace['alignment']:
            # Check if the step corresponds to a 'model move' (indicating a supported transition)
            if step[0] == step[1] and step[1] is not None:
                activated_transitions.add(step[1])
    total_transitions = len(net.transitions)
    support = len(activated_transitions) / total_transitions
    
    # Calculate Coverage (fraction of traces covered by at least one activated transition)
    covered_traces = sum(1 for trace in aligned_traces if any(step[0] == step[1] and step[1] is not None for step in trace['alignment']))
    total_traces = len(log)
    coverage = covered_traces / total_traces

    return support, coverage


def evaluate_routines(log, net, im, fm, token_base, routine_label, routines):
    gt_routine_activities = extract_transitions(net)
    pattern_activities = set(routines[routine_label])

    case_id = log['case:concept:name'].unique()[0]
    routine_length = len(log[log['case:concept:name'] == case_id])

    # Calculate the Jaccard Coefficient (Intersection over Union)
    intersection = pattern_activities.intersection(gt_routine_activities)
    union = pattern_activities.union(gt_routine_activities)
    jc = len(intersection) / len(union)

    log = pm4py.convert_to_event_log(log)
    support, coverage = token_base_evaluation(log, net, im, fm) if token_base else alignment_base_evaluation(log, net, im, fm)

    # Calculate fitness, precision, generalization, and F-score
    q_o = algorithm.apply(log, net, im, fm)
    fitness = round(q_o['fitness']['average_trace_fitness'],3)
    prec = round(q_o['precision'],3)
    gen = round(q_o['generalization'],3)
    simp = round(q_o['simplicity'],3)
    f_score = get_fScore(fitness, prec)

    # Print metrics
    # print("\nEvaluation Scores:")
    # print("=====================")
    # print("Fitness: ", fitness)
    # print("Precision: ", prec)
    # print("Generalization: ", gen)
    # print("Simplicity: ", simp)
    
    # print(f"\nCoverage: {coverage:.2f}")
    # print(f"Support: {support:.2f}")
    # print(f"Jaccard Coefficient: {jc:.2f}")

    return [routine_label, len(log), len(routines[routine_label]), fitness, prec, gen, simp, f_score, coverage, support, jc]

## *Routine by Routine Evaluation*

In [9]:
# Iterate over traces and process activities
def penalize_trace(trace, pattern_activities):
    trace['concept:name'] = trace['concept:name'].apply(
        lambda x: f"extra_{x}" if x not in pattern_activities else x
    )
    return trace


def routines_evaluate(activity_log, routine_vectors, routines):
    key_value = 1
    for pattern_label in routine_vectors.keys():
        fitted_traces = activity_log[activity_log['pattern_label'] == pattern_label]
        if len(fitted_traces):
            JC_scores = {}
            # print("\n")
            # print("="*150)
            for file_name in os.listdir(cpn_ground_truth_dir):
                model_name = file_name.split('.')[0]
                cpn_ground_truth_path = os.path.join(cpn_ground_truth_dir, file_name)
                gt_net, gt_im, gt_fm = pnml_importer.import_net(cpn_ground_truth_path)
                JC = get_JC(gt_net, gt_im, gt_fm, routine_label=pattern_label, routines=routines)
                JC_scores[file_name] = JC
                # print(f"JC Score of {pattern_label} is {JC} with {model_name}")
                # print("="*150)
                # print("\n")
            # Find the key with the maximum value
            max_key = max(JC_scores, key=JC_scores.get)
            max_value = JC_scores[max_key]
        
            # Output the result
            # print(f"\nMaximum JC Score of {pattern_label} is {max_value}")
    
            cpn_ground_truth_path = os.path.join(cpn_ground_truth_dir, max_key)
            gt_net, gt_im, gt_fm = pnml_importer.import_net(cpn_ground_truth_path)
            scores = evaluate_routines(fitted_traces, gt_net, gt_im, gt_fm, token_base=False, routine_label=pattern_label, routines=routines)
            result_dic[key_value] = scores
            key_value += 1
        else:
            JC_scores = {}
            # print("\n")
            # print("="*150)
            for file_name in os.listdir(cpn_ground_truth_dir):
                model_name = file_name.split('.')[0]
                cpn_ground_truth_path = os.path.join(cpn_ground_truth_dir, file_name)
                gt_net, gt_im, gt_fm = pnml_importer.import_net(cpn_ground_truth_path)
                JC = get_JC(gt_net, gt_im, gt_fm, routine_label=pattern_label, routines=routines)
                JC_scores[file_name] = JC
                # print(f"JC Score of {pattern_label} is {JC} with {model_name}")
                # print("="*150)
                # print("\n")
    
            # Find the key with the maximum value
            max_key = max(JC_scores, key=JC_scores.get)
            max_value = JC_scores[max_key]
        
            # Output the result
            # print(f"\nMaximum JC Score of {pattern_label} is {max_value}")
            
            scores = [pattern_label, 0, len(routines[pattern_label]), 0, 0, 0, 0, 0, 0, 0, max_value]
            result_dic[key_value] = scores
            key_value += 1
    return result_dic

## *Summary of Evaluation Scores*

In [10]:
# ✅ Function to calculate F-score given fitness and precision
def calculate_f_score(fitness, precision):
    if fitness + precision == 0:
        return 0
    return 2 * (fitness * precision) / (fitness + precision)

# ✅ Function to calculate simple and weighted averages for a single result dictionary
def UI_logs_summary(result_dic):
    # Column names including Coverage, Support, and JC
    columns = ["Metrics", "Traces", "Trace Length", "Fitness", "Precision", 
               "Generalization", "Simplicity", "F-Score", "Coverage", "Support", "JC"]

    # Initialize lists for averages
    simple_average_values = ["Simple Average"]
    weighted_average_values = ["Weighted Average"]

    total_traces = sum(result_dic[key][1] for key in result_dic if isinstance(key, int))
    simple_average_values.append(total_traces)
    weighted_average_values.append(total_traces)

    mean_fitness = mean_precision = weighted_fitness = weighted_precision = 0

    # Iterate through metrics starting from index 2 ("Trace Length")
    for i in range(2, len(result_dic[1])):  
        values = [result_dic[key][i] for key in result_dic if isinstance(key, int)]

        simple_avg = sum(values) / len(values)
        weighted_avg = sum(result_dic[key][i] * result_dic[key][1] for key in result_dic if isinstance(key, int)) / total_traces

        simple_avg = round(simple_avg, 3)
        weighted_avg = round(weighted_avg, 3)
        
        if i == 3:  # Fitness column
            mean_fitness = simple_avg
            weighted_fitness = weighted_avg
        elif i == 4:  # Precision column
            mean_precision = simple_avg
            weighted_precision = weighted_avg

        simple_average_values.append(simple_avg)
        weighted_average_values.append(weighted_avg)

    # 🔢 Calculate and update the F-Score
    f_score_simple_avg = calculate_f_score(mean_fitness, mean_precision)
    f_score_weighted_avg = calculate_f_score(weighted_fitness, weighted_precision)

    # Update F-Score at the correct index (7th metric after 'Metrics' & 'Traces')
    simple_average_values[7] = round(f_score_simple_avg, 3)
    weighted_average_values[7] = round(f_score_weighted_avg, 3)

    # ➕ Insert average rows into result_dic
    next_key_simple = max([key for key in result_dic if isinstance(key, int)], default=0) + 1
    next_key_weighted = next_key_simple + 1

    result_dic[next_key_simple] = simple_average_values
    result_dic[next_key_weighted] = weighted_average_values

    # 📝 Convert to DataFrame and print
    result_df = pd.DataFrame.from_dict(result_dic, orient='index', columns=columns)
    result_df = result_df.set_index('Metrics')

    # print("\n✅ Per-Log Summary:")
    # print(tabulate(result_df, headers='keys', tablefmt='psql'))

    return result_dic, simple_average_values  # Return simple average row for aggregation

# 🚀 Function to handle cross-validation and final aggregation
def cross_validation_summary(cross_val_results):
    """Performs cross-validation, collects simple averages, and calculates final averages."""
    iter_results = {}
    simple_averages_list = []

    # 📊 Collect simple averages from each iteration
    for i, result_dic in enumerate(cross_val_results, start=1):
        _, simple_avg_row = UI_logs_summary(result_dic)
        iter_results[f"CV {i}"] = simple_avg_row[1:]  # Exclude 'Simple Average' label
        simple_averages_list.append(simple_avg_row[1:])

    # 🔎 Convert iteration results into DataFrame
    iter_columns = ["Traces", "Trace Length", "Fitness", "Precision", "Gen", 
                    "Simp", "F-Score", "Coverage", "Support", "JC"]
    iter_df = pd.DataFrame(iter_results, index=iter_columns).T

    print("\n✅ Cross-Validation Iteration Results:")
    print(tabulate(iter_df, headers='keys', tablefmt='psql'))

    # 🧮 Final Simple and Weighted Averages
    final_simple_avg = iter_df.mean().tolist()
    total_traces = iter_df["Traces"].sum()
    weighted_avg = [(iter_df[col] * iter_df["Traces"]).sum() / total_traces for col in iter_df.columns]

    # 📢 Final Aggregated Results
    final_df = pd.DataFrame(
        [final_simple_avg, weighted_avg],
        index=["Simple Average", "Weighted Average"],
        columns=iter_columns
    )

    print("\n🚀 Final Aggregated Results:")
    print(tabulate(final_df, headers='keys', tablefmt='outline'))

    return iter_df, final_df

In [11]:
# # Initialize two lists for average values
# simple_average_values = ["Simple Average"]
# weighted_average_values = ["Weighted Average"]

# # Calculate the sum of Traces for weighted calculations
# total_traces = sum(result_dic[key][1] for key in result_dic if isinstance(key, int))

# # Append the total traces to both average rows
# simple_average_values.append(total_traces)  # Sum of Traces for simple average
# weighted_average_values.append(total_traces)  # Sum of Traces for weighted average

# # Calculate averages for each metric column
# for i in range(2, len(result_dic[1])):  # Start from index 2 to skip Routine and Traces
#     values = [result_dic[key][i] for key in result_dic if isinstance(key, int)]

#     if i == 2:
#         # Special case: Simple average for trace length (index 2)
#         simple_avg = sum(values) / len(values)
#         weighted_avg = simple_avg  # Use the same for weighted average
#     else:
#         # Simple average for other metrics
#         simple_avg = sum(values) / len(values)
        
#         # Weighted average for other metrics
#         weighted_avg = sum(result_dic[key][i] * result_dic[key][1] for key in result_dic if isinstance(key, int)) / total_traces

#     # Append the averages to the respective lists
#     simple_average_values.append(simple_avg)
#     weighted_average_values.append(weighted_avg)

# # Find the next available numeric keys
# next_key_simple = max([key for key in result_dic if isinstance(key, int)]) + 1
# next_key_weighted = next_key_simple + 1

# # Add the new rows to result_dic
# result_dic[next_key_simple] = simple_average_values
# result_dic[next_key_weighted] = weighted_average_values


In [12]:
# # Initialize two lists for average values
# simple_average_values = ["Simple Average"]
# weighted_average_values = ["Weighted Average"]

# # Calculate the sum of Traces for weighted calculations
# total_traces = sum(result_dic[key][1] for key in result_dic if isinstance(key, int))

# # Append the total traces to both average rows
# simple_average_values.append(total_traces)  # Sum of Traces for simple average
# weighted_average_values.append(total_traces)  # Sum of Traces for weighted average

# # Function to calculate F-score given fitness and precision
# def calculate_f_score(fitness, precision):
#     if fitness + precision == 0:
#         return 0
#     return 2 * (fitness * precision) / (fitness + precision)

# # Store mean fitness and precision for later F-score calculation
# mean_fitness = 0
# mean_precision = 0
# weighted_fitness = 0
# weighted_precision = 0

# # Calculate averages for each metric column
# for i in range(2, len(result_dic[1])):  # Start from index 2 to skip Routine and Traces
#     values = [result_dic[key][i] for key in result_dic if isinstance(key, int)]

#     if i == 2:
#         # Special case: Simple average for trace length (index 2)
#         simple_avg = sum(values) / len(values)
#         weighted_avg = simple_avg  # Use the same for weighted average
#     else:
#         # Simple average for other metrics
#         simple_avg = sum(values) / len(values)
        
#         # Weighted average for other metrics
#         weighted_avg = sum(result_dic[key][i] * result_dic[key][1] for key in result_dic if isinstance(key, int)) / total_traces

#     # Store mean fitness and precision for F-score calculation
#     if i == 3:  # Fitness column index
#         mean_fitness = simple_avg
#         weighted_fitness = weighted_avg
#     elif i == 4:  # Precision column index
#         mean_precision = simple_avg
#         weighted_precision = weighted_avg

#     # Append the averages to the respective lists
#     simple_average_values.append(simple_avg)
#     weighted_average_values.append(weighted_avg)

# # Calculate the F-score using the mean fitness and precision values
# f_score_simple_avg = calculate_f_score(mean_fitness, mean_precision)
# f_score_weighted_avg = calculate_f_score(weighted_fitness, weighted_precision)

# # Replace the F-score column value with calculated F-score
# simple_average_values[7] = f_score_simple_avg  # Assuming F-Score is at index 6
# weighted_average_values[7] = f_score_weighted_avg

# # Find the next available numeric keys
# next_key_simple = max([key for key in result_dic if isinstance(key, int)]) + 1
# next_key_weighted = next_key_simple + 1

# # Add the new rows to result_dic
# result_dic[next_key_simple] = simple_average_values
# result_dic[next_key_weighted] = weighted_average_values

In [13]:
# result_df = pd.DataFrame(result_dic)
# result_df = result_df.set_index('Metrics')
# # displaying the DataFrame
# # print(tabulate(result_df.T, headers = 'keys', tablefmt = 'psql'))
# result_df.T

## *main*

In [14]:




# if noise_prob == 0:
#     log_path = f"Transformed_Logs_and_Results/Our/Transformed_Log_Without_Noise/transform_log{log_number}.xes"
#     routines_file_path = f"Transformed_Logs_and_Results/Domas/Transformed_Log_Without_Noise/Discovered_Routines/domas_noisy_transform_log{log_number}_{extraction_tech_dict[extraction_id]}.txt"
# else:
#     log_path = f"Transformed_Logs_and_Results/Our/Transformed_Log_With_Noise_{noise_prob}/noisy_transform_log{log_number}.xes"
#     routines_file_path = f"Transformed_Logs_and_Results/Domas/Transformed_Log_With_Noise_{noise_prob}/Discovered_Routines/domas_noisy_transform_log{log_number}_{extraction_tech_dict[extraction_id]}.txt"



# key_value = 1
# result_dic = {
#     'Metrics':['Routine', "Traces", "Length", 'Fitness','Precision','Generalization','Simplicity', 'F_Score', 'coverage', 'Support', 'JC'],
# }

In [15]:
extraction_tech_dict = {
    1: "frequency",
    2: "coverage",
    3: "cohesion",
    4: "length",
}

noise_prob = 0.4

for log_number in range(1, 10):
    for extraction_id in range(1, 5):
        cross_val_results = []
        for variant in range(1, 11):
            try:
                cpn_ground_truth_dir = f"GT_Models/log{log_number}/"
                
                log_path = f"Transformed_Logs_and_Results/Our/Transformed_Log_With_Noise_{noise_prob}/log{log_number}/noisy_transform_log{log_number}_{variant}.xes"
                routines_file_path = f"Transformed_Logs_and_Results/Domas/Transformed_Log_With_Noise_{noise_prob}/Discovered_Routines/log{log_number}/domas_noisy_transform_log{log_number}_{variant}.csv_{extraction_tech_dict[extraction_id]}.txt"
                
                result_dic = {
                    'Metrics':['Routine', "Traces", "Length", 'Fitness','Precision','Generalization','Simplicity', 'F_Score', 'coverage', 'Support', 'JC'],
                }
                
                log, reference_set = UI_read_log(log_path=log_path)
                all_routines = read_routines(routines_file_path)
                routine_vectors, routines = get_discover_routines_encoding(all_routines=all_routines, reference_set=reference_set)
                
                session_log = f.create_session(log)
                encoded_log = freq_encoding(session_log)
                
                # Assign labels
                encoded_log = assign_labels_by_distance(encoded_log, routine_vectors, reference_set)
                
                activity_log = assign_pattern_to_traces(session_log, encoded_log)
                result_dict = routines_evaluate(activity_log, routine_vectors, routines)
                cross_val_results.append(result_dic)
            except Exception as e:
                print(f"Error encountered for Log{log_number}, Variant {variant}: {e}")
                continue  # Continue with the next variant
            
        # Run cross-validation summary
        iteration_results, final_averages = cross_validation_summary(cross_val_results)
        results_df = pd.concat([iteration_results, final_averages])
    
        outputfile = f"Transformed_Logs_and_Results/Domas/Transformed_Log_With_Noise_{noise_prob}/Results"
        if not os.path.exists(outputfile):
            os.makedirs(outputfile)
        output_file_name = f"log{log_number}_{extraction_tech_dict[extraction_id]}_reuslts.csv"
        results_df.to_csv(os.path.join(outputfile, output_file_name))

replaying log with TBR, completed variants :: 100%|████████████████████████████████| 978/978 [00:00<00:00, 3384.21it/s]



✅ Cross-Validation Iteration Results:
+-------+----------+----------------+-----------+-------------+-------+--------+-----------+------------+-----------+-------+
|       |   Traces |   Trace Length |   Fitness |   Precision |   Gen |   Simp |   F-Score |   Coverage |   Support |    JC |
|-------+----------+----------------+-----------+-------------+-------+--------+-----------+------------+-----------+-------|
| CV 1  |      100 |              1 |     0.636 |       0.997 | 0.9   |      1 |     0.777 |          1 |     0.929 | 0.077 |
| CV 2  |      100 |              1 |     0.645 |       0.996 | 0.899 |      1 |     0.783 |          1 |     0.929 | 0.077 |
| CV 3  |      100 |              1 |     0.643 |       0.997 | 0.899 |      1 |     0.782 |          1 |     0.929 | 0.077 |
| CV 4  |      100 |              1 |     0.663 |       0.996 | 0.898 |      1 |     0.796 |          1 |     0.929 | 0.077 |
| CV 5  |      100 |              1 |     0.647 |       0.992 | 0.899 |      1 

replaying log with TBR, completed variants :: 100%|██████████████████████████████████| 20/20 [00:00<00:00, 2439.83it/s]



✅ Cross-Validation Iteration Results:
+-------+----------+----------------+-----------+-------------+-------+--------+-----------+------------+-----------+-------+
|       |   Traces |   Trace Length |   Fitness |   Precision |   Gen |   Simp |   F-Score |   Coverage |   Support |    JC |
|-------+----------+----------------+-----------+-------------+-------+--------+-----------+------------+-----------+-------|
| CV 1  |      100 |          3.6   |     0.493 |       0.734 | 0.455 |  0.8   |     0.59  |      0.8   |     0.714 | 0.277 |
| CV 2  |      100 |          3.75  |     0.622 |       0.735 | 0.588 |  1     |     0.674 |      1     |     0.857 | 0.269 |
| CV 3  |      100 |          3.2   |     0.51  |       0.734 | 0.512 |  0.8   |     0.602 |      0.8   |     0.743 | 0.246 |
| CV 4  |      100 |          3.5   |     0.633 |       0.935 | 0.623 |  1     |     0.755 |      1     |     0.929 | 0.269 |
| CV 5  |      100 |          3.75  |     0.608 |       0.913 | 0.657 |  1     

replaying log with TBR, completed variants :: 100%|██████████████████████████████████| 18/18 [00:00<00:00, 2578.38it/s]


...


✅ Cross-Validation Iteration Results:
+-------+----------+----------------+-----------+-------------+-------+--------+-----------+------------+-----------+-------+
|       |   Traces |   Trace Length |   Fitness |   Precision |   Gen |   Simp |   F-Score |   Coverage |   Support |    JC |
|-------+----------+----------------+-----------+-------------+-------+--------+-----------+------------+-----------+-------|
| CV 1  |     2000 |              3 |     0.534 |       0.806 | 0.861 |      1 |     0.642 |          1 |     0.909 | 0.273 |
| CV 2  |     2000 |              5 |     0.555 |       0.812 | 0.882 |      1 |     0.659 |          1 |     0.909 | 0.455 |
| CV 3  |     2000 |              5 |     0.553 |       0.801 | 0.882 |      1 |     0.654 |          1 |     0.909 | 0.455 |
| CV 4  |     2000 |              5 |     0.554 |       0.818 | 0.882 |      1 |     0.661 |          1 |     0.909 | 0.455 |
| CV 5  |     2000 |              5 |     0.559 |       0.796 | 0.882 |      1 

parsing log, completed traces :: 100%|████████████████████████████████████████████| 2000/2000 [00:05<00:00, 344.92it/s]


Error encountered for Log9, Variant 4: operands could not be broadcast together with shapes (2000,1,47) (0,) 


replaying log with TBR, completed variants :: 100%|████████████████████████████| 18550/18550 [00:04<00:00, 3744.05it/s]



✅ Cross-Validation Iteration Results:
+------+----------+----------------+-----------+-------------+-------+--------+-----------+------------+-----------+-------+
|      |   Traces |   Trace Length |   Fitness |   Precision |   Gen |   Simp |   F-Score |   Coverage |   Support |    JC |
|------+----------+----------------+-----------+-------------+-------+--------+-----------+------------+-----------+-------|
| CV 1 |     2000 |          5     |     0.503 |       0.684 | 0.736 |    1   |     0.58  |        1   |     0.818 | 0.455 |
| CV 2 |     2000 |          6     |     0.555 |       0.812 | 0.882 |    1   |     0.659 |        1   |     0.909 | 0.545 |
| CV 3 |     2000 |          6     |     0.553 |       0.801 | 0.882 |    1   |     0.654 |        1   |     0.909 | 0.545 |
| CV 4 |     2000 |          6.5   |     0.28  |       0.398 | 0.441 |    0.5 |     0.329 |        0.5 |     0.455 | 0.545 |
| CV 5 |     2000 |          5     |     0.473 |       0.397 | 0.738 |    1   |     0.